In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
course = pd.read_csv('course.csv')
problem = pd.read_csv('problemstate.csv')

In [3]:
course.drop(columns='courseID', axis=1, inplace=True)

In [4]:
# periksa sampel teks
def sample_text(df, idx):
    for col in df.columns:
        print(f'{col}:')
        print(df[col].iloc[idx])
        print('\n')
        
sample_text(course, 3)

judul:
Indonesia Sustainability Coral Reef University Network (ISCORE)


deskripsi:
Terumbu karang menutupi kurang dari 1% wilayah lautan, tetapi mereka secara langsung mendukung jutaan orang dengan
menyediakan makanan, pekerjaan, dan sumber daya lainnya. Bahkan lebih banyak orang yang mendapat manfaat dari
terumbu karang secara tidak langsung; ekosistem ini membantu memberi makan 1 miliar orang di Asia saja.
Populasi manusia hampir 7 miliar orang, dan kemungkinan akan tumbuh menjadi 9 miliar pada tahun 2040. Kita
membutuhkan terumbu karang (dan ekosistem lainnya) untuk memasok lebih banyak sumber daya untuk mendukung
jumlah kita yang terus bertambah, tetapi mereka semakin terancam dengan kehancuran .


dampak:
pelindungan habibat; mengurangi polusi; mengurangi dampak erosi; promosi pariwisata; meningkatkan komunitas lokal; pemeliharaan agen perubahan berkelanjutan dan perlindungan terumbu karang


modul:
Sustainability Leadership; Coral Reef Ecology; Human threats & Challenges; Integr

In [5]:
sample_text(problem, 0)

desa:
desa pinogu


problem:
Budidaya dan pengolahan kopi


lokasi:
gorontalo




In [6]:
# agregat tiap masalah pada 1 desa menjadi list dalam 1 baris
problem_agg = (problem.groupby(['desa','lokasi'])
          .agg({'problem': lambda x: ", ".join(x)})
          .reset_index())

problem_agg = problem_agg.assign(loc_desa = problem_agg['desa'] + ', ' + problem_agg['lokasi'])
problem_agg = problem_agg.drop(columns=['desa','lokasi'], axis=1)

In [7]:
sample_text(problem_agg, 0)

problem:
Fasilitas penunjang wisata yang masih minim, Wisata kuliner yang terbatas, Akses informasi wisata yang masih terbatas, Faslitas penunjang yang masih terbatas, Belum ada inovasi pengelolaan SDA, seperti makanan sebagai wisata kuliner., Sumber daya manusia yang rendah untuk mengembangkan potensia yang ada.


loc_desa:
desa aimoli, alor




In [8]:
# menggabungkan deskripsi dan dampak pada course menjadi 1 kolom
course2 = course.assign(summary = course['deskripsi'] + ', ' + course['dampak'] + ', ' + course['modul'])
course2 = course2.drop(columns=['deskripsi','dampak', 'modul'], axis=1)

sample_text(course2, 3)

judul:
Indonesia Sustainability Coral Reef University Network (ISCORE)


summary:
Terumbu karang menutupi kurang dari 1% wilayah lautan, tetapi mereka secara langsung mendukung jutaan orang dengan
menyediakan makanan, pekerjaan, dan sumber daya lainnya. Bahkan lebih banyak orang yang mendapat manfaat dari
terumbu karang secara tidak langsung; ekosistem ini membantu memberi makan 1 miliar orang di Asia saja.
Populasi manusia hampir 7 miliar orang, dan kemungkinan akan tumbuh menjadi 9 miliar pada tahun 2040. Kita
membutuhkan terumbu karang (dan ekosistem lainnya) untuk memasok lebih banyak sumber daya untuk mendukung
jumlah kita yang terus bertambah, tetapi mereka semakin terancam dengan kehancuran ., pelindungan habibat; mengurangi polusi; mengurangi dampak erosi; promosi pariwisata; meningkatkan komunitas lokal; pemeliharaan agen perubahan berkelanjutan dan perlindungan terumbu karang, Sustainability Leadership; Coral Reef Ecology; Human threats & Challenges; Integrating Sustainabilit

In [9]:
course_clean = course2.copy()
prob_clean = problem_agg.copy()

In [10]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# variable to clear symbols
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')

# stopword
f = open("tala_stopword.txt", "r")
stopword_list = []
for line in f:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    stopword_list.append(line_list[0])
f.close()

len(stopword_list)

705

In [11]:
def clean_text(text):
    text = stemmer.stem(text)
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopword_list) # hapus stopword dari kolom deskripsi
    return text

In [12]:
course_clean['summary'] = course_clean['summary'].apply(clean_text)
# course_clean['modul'] = course_clean['modul'].apply(clean_text)
prob_clean['problem'] = problem_agg['problem'].apply(clean_text)

In [13]:
sample_text(course_clean, 3)

judul:
Indonesia Sustainability Coral Reef University Network (ISCORE)


summary:
terumbu karang tutup 1 wilayah laut langsung dukung juta orang sedia makan kerja sumber daya bahkan banyak orang manfaat terumbu karang langsung ekosistem bantu makan 1 miliar orang asia populasi manusia 7 miliar orang akan tumbuh 9 miliar 2040 butuh terumbu karang ekosistem pasok banyak sumber daya dukung ancam hancur lindung habibat polusi dampak erosi promosi pariwisata tingkat komunitas lokal pelihara agen ubah lindung terumbu karang sustainability leadership coral reef ecology human threats challenges integrating sustainability in coral reef improving reef management by sustainable financing for communities benefit




In [14]:
sample_text(prob_clean, 0)

problem:
fasilitas tunjang wisata minim wisata kuliner batas akses informasi wisata batas faslitas tunjang batas ada inovasi kelola sda makan bagai wisata kuliner sumber daya manusia rendah kembang potensia ada


loc_desa:
desa aimoli, alor




## TF-IDF

In [15]:
# # mengubah kalimat menjadi angka berupa matriks
# def conv_to_tf_idf(df, set_to_idx, col_transform):
#     df.set_index(set_to_idx, inplace=True)
#     tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0)
#     tfidf_matrix = tf.fit_transform(col_transform)
#     # cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#     return tfidf_matrix

# tfidf_course = conv_to_tf_idf(course_clean, 'judul', course_clean['summary'])
# tfidf_prob = conv_to_tf_idf(prob_clean, 'loc_desa', prob_clean['problem'])

In [16]:
course_clean.set_index('judul', inplace=True)
prob_clean.set_index('loc_desa', inplace=True)

In [17]:
course_clean

,summary
judul,
Fundamental Course (1),fundamental attitude toward sustainability uba...
Fundamental Course (2),innovation method as problem solving paham kon...
Fundamental Course (3),the way to build sustainable business kenal su...
Indonesia Sustainability Coral Reef University Network (ISCORE),terumbu karang tutup 1 wilayah laut langsung d...
Ecotourism,iring longgar jalan stabilitas bijak jalan tum...
Moringa Academy,moringa oliefera kelor rupa salah tanam pusaka...
Indonesia Sustainable Social Forestry Education Program (IS-FREE),program kait potensi hutan sosial hutan produk...
Waste Management,kota darurat sampah desa desa indonesia milik ...
Integrated Farming,sistem tani padu rupa sistem integrasi tani ga...


In [18]:
prob_clean

,problem
loc_desa,
"desa aimoli, alor",fasilitas tunjang wisata minim wisata kuliner ...
"desa dataran hijau, gorontalo",akses sulit infrastruktur kendala listrik kend...
"desa lewalu, alor",sulit cari souvenir atau sampah akses jalan ba...
"desa pinogu, gorontalo",budidaya olah kopi olah produk sapi akses jala...
"desa saleman, maluku",budaya masyarakat buang sampah laut etika masy...


### Modelling

In [19]:
def find_nearest_similarity(dataset1, dataset2):
    # Extract text and index from DataFrame
    text1 = dataset1['summary'].tolist()
    text2 = dataset2['problem'].tolist()
    index1 = dataset1.index.tolist()
    index2 = dataset2.index.tolist()

    # Vectorize the datasets using TF-IDF
    vectorizer = TfidfVectorizer()

    vectors1 = vectorizer.fit_transform(text1)
    vectors2 = vectorizer.transform(text2)

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(vectors1, vectors2)

    # Find the nearest similarity for each row in dataset1
    nearest_similarities = {}
    nearest_indices = {}

    for i in range(len(dataset1)):
        max_similarity = -1
        max_index = -1

        for j in range(len(dataset2)):
            if similarity_matrix[i, j] > max_similarity:
                max_similarity = similarity_matrix[i, j]
                max_index = index2[j]

        nearest_similarities[index1[i]] = max_similarity
        nearest_indices[index1[i]] = max_index

    return nearest_indices, nearest_similarities

def get_top_3_similar_indices(index1, similarity_matrix):
    similarities = similarity_matrix[index1]
    sorted_indices = similarities.argsort()[::-1][:3]

    return sorted_indices.tolist()

In [20]:
nearest_indices, nearest_similarities = find_nearest_similarity(course_clean, prob_clean)

In [23]:
for idx in course_clean.index:
    print("Nearest similarity pair for [{}]: [{}], Similarity: {}".format(idx, nearest_indices[idx], nearest_similarities[idx]))

Nearest similarity pair for [Fundamental Course (1)]: [desa dataran hijau, gorontalo], Similarity: 0.09511561561872749
Nearest similarity pair for [Fundamental Course (2)]: [desa dataran hijau, gorontalo], Similarity: 0.08373259676577054
Nearest similarity pair for [Fundamental Course (3)]: [desa dataran hijau, gorontalo], Similarity: 0.012214240866418397
Nearest similarity pair for [Indonesia Sustainability Coral Reef University Network (ISCORE)]: [desa pinogu, gorontalo], Similarity: 0.1501260001545122
Nearest similarity pair for [Ecotourism]: [desa aimoli, alor], Similarity: 0.2610730142189104
Nearest similarity pair for [Moringa Academy]: [desa saleman, maluku], Similarity: 0.12957921349313437
Nearest similarity pair for [Indonesia Sustainable Social Forestry Education Program (IS-FREE)]: [desa saleman, maluku], Similarity: 0.1958304065966555
Nearest similarity pair for [Waste Management]: [desa lewalu, alor], Similarity: 0.40372355966887286
Nearest similarity pair for [Integrated 